In [1]:
from tinygrad.densetensor import DenseTensor, cl_queue
from tinygrad.sparsetensor import SparseTensor
import numpy as np
import tinygrad.optim as optim
from extra.utils import get_parameters


%load_ext autoreload
%autoreload 2

DEVICE:GPU


In [2]:
dim1 = 6
dim2 = 8
dim3 = 4

np.random.seed(9)

In [3]:
x_init = np.random.randn(dim1,dim2).astype(np.float32)
w_init = np.random.randn(dim2,dim3).astype(np.float32)
x = DenseTensor(x_init)
y = DenseTensor(np.random.randn(dim1,dim3))

warning, (6, 4) isn't float32, it's float64


W_TEST = SparseTensor.uniform(dim2,dim3, randsparsity=0.6).to_numpy()
W_TEST

In [4]:
W_TEST = SparseTensor(w_init).to_numpy()
W_TEST

array([[-6.38729751e-01,  8.24770331e-01, -1.21083879e+00,
        -5.03405392e-01],
       [-7.01915681e-01, -1.97427106e+00, -2.65573215e+00,
        -5.76822497e-02],
       [-6.56186581e-01, -6.61706686e-01,  7.69348443e-01,
        -8.99004877e-01],
       [ 1.69363797e+00, -1.69733524e+00, -2.79337025e+00,
        -2.26150647e-01],
       [ 3.97428840e-01,  1.65970361e+00, -4.93746817e-01,
        -3.76097679e-01],
       [-1.69739768e-01,  2.41710639e+00, -1.80884051e+00,
         3.39751154e-01],
       [-2.27297600e-02, -9.59997058e-01, -3.83114427e-01,
         1.09529994e-01],
       [-8.55162859e-01,  2.21606664e-04,  6.63855076e-01,
         7.49480963e-01]])

In [5]:
class MLP:
  def __init__(self, init_dense):
    #self.W = DenseTensor(init_dense)
    self.W = DenseTensor(W_TEST)

  def parameters(self):
    return get_parameters(self)

  def forward(self, x):
    out = x.dot(self.W)
    out = out.softmax()
    #out = out.mul(self.m).add(self.m).sum()
    return out

In [6]:
class MLP2:
  def __init__(self):
    self.W = SparseTensor(W_TEST)
    #self.W = SparseTensor.uniform(dim2,dim3)
    #self.W = SparseTensor.uniform(dim2,dim3)

  def parameters(self):
    return get_parameters(self)

  def forward(self, x):
    out = x.dot(self.W)
    out = out.softmax()
    #out = out.mul(self.m).add(self.m).sum()
    return out

In [7]:
def loss_fn(y, y_pred):
    return ((y-y_pred)**2)**.5

In [8]:
model2 = MLP2()

In [9]:
model2.W.get_nnzs()

array([4, 4, 4, 4, 4, 4, 4, 4], dtype=uint32)

In [10]:
dense_init = model2.W.to_numpy()
dense_init

array([[-6.38729751e-01,  8.24770331e-01, -1.21083879e+00,
        -5.03405392e-01],
       [-7.01915681e-01, -1.97427106e+00, -2.65573215e+00,
        -5.76822497e-02],
       [-6.56186581e-01, -6.61706686e-01,  7.69348443e-01,
        -8.99004877e-01],
       [ 1.69363797e+00, -1.69733524e+00, -2.79337025e+00,
        -2.26150647e-01],
       [ 3.97428840e-01,  1.65970361e+00, -4.93746817e-01,
        -3.76097679e-01],
       [-1.69739768e-01,  2.41710639e+00, -1.80884051e+00,
         3.39751154e-01],
       [-2.27297600e-02, -9.59997058e-01, -3.83114427e-01,
         1.09529994e-01],
       [-8.55162859e-01,  2.21606664e-04,  6.63855076e-01,
         7.49480963e-01]])

In [11]:
model = MLP(dense_init)

### Dense

In [12]:
iters = 1
optimizer = optim.SGD(model.parameters(), lr=1)

In [13]:
for i in range(iters):
    res = model.forward(x)
    optimizer.zero_grad()
    loss = loss_fn(res, y)
    loss.backward()
    optimizer.step()

SHAPES: (6, 8) (8, 4)
INPUT [[ 1.1085547e-03 -2.8954408e-01 -1.1160663e+00 -1.2882757e-02
  -3.7836146e-01 -4.8113537e-01 -1.5173311e+00 -4.9087200e-01]
 [-2.4068058e-01 -6.4794743e-01  6.3589108e-01  1.7401173e+00
   2.9668221e-01  7.0750368e-01  1.8228158e+00  4.3076903e-01]
 [ 1.5427296e+00 -9.0072119e-01 -1.3712502e-01  1.2975791e+00
   6.7527115e-01  3.1958118e-02  9.1814590e-01  3.8050947e-01]
 [ 5.1636750e-01 -3.5523945e-01  2.0877700e-01  3.2841107e-01
  -4.9822477e-01 -2.0917768e+00 -8.2587741e-02  2.4551826e+00]
 [-2.6721101e+00 -9.1327929e-01 -2.2731435e-01  2.6931539e-01
   1.1304612e+00  1.0423975e+00  1.3038105e+00  1.3894007e+00]
 [-6.5645260e-01 -5.6257296e-02 -4.9990261e-01  4.3641937e-01
  -3.7581304e-01 -9.2306161e-01  1.9172503e+00 -1.5030284e-01]]
OUT: [[ 1.50174284e-02 -4.03571635e-01  1.78856969e-01 -1.44680411e-01
   1.92041516e-01 -9.14528146e-02 -1.23737752e-01 -1.98523298e-01]
 [-2.69819312e-02  2.87787933e-02 -2.76422128e-03  7.13188201e-02
  -2.35424917e-02

NameError: name 'asdf' is not defined

In [ ]:
model.parameters()

In [ ]:
res.cpu().data

In [ ]:
loss.cpu().data

In [ ]:
res.grad.cpu().data

In [ ]:
model.W.grad.cpu().data

### Second

In [14]:
w_init

array([[-6.38729751e-01,  8.24770331e-01, -1.21083879e+00,
        -5.03405392e-01],
       [-7.01915681e-01, -1.97427106e+00, -2.65573215e+00,
        -5.76822497e-02],
       [-6.56186581e-01, -6.61706686e-01,  7.69348443e-01,
        -8.99004877e-01],
       [ 1.69363797e+00, -1.69733524e+00, -2.79337025e+00,
        -2.26150647e-01],
       [ 3.97428840e-01,  1.65970361e+00, -4.93746817e-01,
        -3.76097679e-01],
       [-1.69739768e-01,  2.41710639e+00, -1.80884051e+00,
         3.39751154e-01],
       [-2.27297600e-02, -9.59997058e-01, -3.83114427e-01,
         1.09529994e-01],
       [-8.55162859e-01,  2.21606664e-04,  6.63855076e-01,
         7.49480963e-01]], dtype=float32)

In [15]:
model2 = MLP2()

In [16]:
optimizer2 = optim.SGD(model2.parameters(), lr=1)

In [17]:
for i in range(iters):
    res2 = model2.forward(x)
    optimizer2.zero_grad()
    loss2 = loss_fn(res2, y)
    loss2.backward()
    optimizer2.step()

INPUT [[ 1.1085547e-03 -2.8954408e-01 -1.1160663e+00 -1.2882757e-02
  -3.7836146e-01 -4.8113537e-01 -1.5173311e+00 -4.9087200e-01]
 [-2.4068058e-01 -6.4794743e-01  6.3589108e-01  1.7401173e+00
   2.9668221e-01  7.0750368e-01  1.8228158e+00  4.3076903e-01]
 [ 1.5427296e+00 -9.0072119e-01 -1.3712502e-01  1.2975791e+00
   6.7527115e-01  3.1958118e-02  9.1814590e-01  3.8050947e-01]
 [ 5.1636750e-01 -3.5523945e-01  2.0877700e-01  3.2841107e-01
  -4.9822477e-01 -2.0917768e+00 -8.2587741e-02  2.4551826e+00]
 [-2.6721101e+00 -9.1327929e-01 -2.2731435e-01  2.6931539e-01
   1.1304612e+00  1.0423975e+00  1.3038105e+00  1.3894007e+00]
 [-6.5645260e-01 -5.6257296e-02 -4.9990261e-01  4.3641937e-01
  -3.7581304e-01 -9.2306161e-01  1.9172503e+00 -1.5030284e-01]] 8 4
OUT: [[ 1.50174284e-02 -4.03571635e-01  1.78856969e-01 -1.44680411e-01
   1.92041516e-01 -9.14528146e-02 -1.23737752e-01 -1.98523298e-01]
 [-2.69819312e-02  2.87787933e-02 -2.76422128e-03  7.13188201e-02
  -2.35424917e-02 -4.62456383e-02  

NameError: name 'asdf' is not defined

#### fwd

In [ ]:
res2.cpu().data==res.cpu().data

In [ ]:
res2.cpu().data

In [ ]:
res.cpu().data - res2.cpu().data

#### loss

In [ ]:
loss.cpu().data==loss2.cpu().data

In [ ]:
loss2.cpu().data

In [ ]:
loss.cpu().data - loss2.cpu().data

#### grad

In [ ]:
res.grad.cpu().data==res2.grad.cpu().data

In [ ]:
res2.grad.cpu().data

In [ ]:
res.grad.cpu().data-res2.grad.cpu().data

#### weight

In [ ]:
def to_dense(data, cols, nnzs, ellw, shape):
    out = np.zeros(shape)
    for row in range(shape[0]):
        for icol in range(nnzs[row]):
            #print('idx:',row,cols[row*ellw+icol])
            out[row,cols[row*ellw+icol]] = data[row*ellw+icol]
    return out

In [ ]:
model.W.grad, model2.W.grad

In [ ]:
model.W.grad.cpu().data

In [ ]:
model2.W.grad.data.cl

In [ ]:
import pyopencl as cl

dim = np.prod(model2.W.grad.shape)
data = np.zeros(dim).astype(np.float32)
cl.enqueue_copy(cl_queue, data, model2.W.grad.datat.cl)
cols = np.zeros(dim).astype(np.uint32)
cl.enqueue_copy(cl_queue, cols, model2.W.grad.idxst.cl)
nnzs = np.zeros(model.W.grad.shape[0]).astype(np.uint32)
cl.enqueue_copy(cl_queue, nnzs, model2.W.grad.nnzst.cl)

In [ ]:
to_dense(data, cols, nnzs, model2.W.grad.ellwt, model2.W.grad.shape)

In [ ]:
data

In [ ]:
cols

In [ ]:
nnzs

In [ ]:
denserec = model.W.cpu().data 
denserec

In [ ]:
sparserec = model2.W.to_numpy()
sparserec

In [ ]:
denserec - sparserec

In [ ]:
sparserec2 = model2.W.to_numpy(dual=True)
sparserec2.T

In [ ]:
denserec - sparserec2.T

In [ ]:
seedmat - denserec